In [1]:
import joblib
import pandas as pd


# Load the saved model, scaler, encoder, and top 5 features
svm_model_top5 = joblib.load('svm_model_top5.pkl')
scaler = joblib.load('scaler.pkl')
top_5_features = joblib.load('top_5_features (1).pkl')
numerical_cols = joblib.load('numerical_cols.pkl')



In [2]:
top_5_features

['LDH', 'IL6', 'Neutrophil %', 'LCR', 'severe_1']

In [3]:
LDH = float(input("Enter LDH level: "))
IL6 = float(input("Enter IL6 level: "))
Neutrophil_percentage = float(input("Enter Neutrophil percentage: "))
LCR = float(input("Enter LCR level: "))
severe_1 = input("Enter severe_1 (categorical value): ")



Enter LDH level: 15
Enter IL6 level: 25
Enter Neutrophil percentage: 60
Enter LCR level: 1.5
Enter severe_1 (categorical value): 1


In [4]:


# Create a DataFrame for the new patient
new_patient_data = {
    'LDH': [LDH],
    'IL6': [IL6],
    'Neutrophil %': [Neutrophil_percentage],
    'LCR': [LCR],
}

# Convert the dictionary to a DataFrame
new_patient_df = pd.DataFrame(new_patient_data)

# Include missing features with default values (e.g., 0 or median)
for col in numerical_cols:
    if col not in new_patient_data:
        new_patient_data[col] = [0]  # or any other default value

new_patient_df = pd.DataFrame(new_patient_data)

# Standardize numerical columns
new_patient_df[numerical_cols] = scaler.transform(new_patient_df[numerical_cols])

# categorical column
new_patient_severe = pd.DataFrame(list(severe_1))
new_patient_severe.columns = ['severe_1']
new_patient_df = pd.concat([new_patient_df, new_patient_severe], axis=1)

# Select the top 5 features
new_patient_df_top5 = new_patient_df[top_5_features]
new_patient_df_top5


,LDH,IL6,Neutrophil %,LCR,severe_1
0,-1.344682,-0.513581,-1.679092,-0.259833,1


In [5]:

# Make a prediction using the loaded model
prediction = svm_model_top5.predict(new_patient_df_top5)
prediction_proba = svm_model_top5.predict_proba(new_patient_df_top5)

# Display the prediction
prediction_1 = "Patient will probably Survive" if prediction[0] == 0 else "Patient may not Survive"
print("Prediction:", prediction_1)

# Display the probability of each class
print("Probability of Survival (Percent):", round(prediction_proba[0][0]*100, 1))


Prediction: Patient will probably Survive
Probability of Survival (Percent): 82.1


## with dataset

In [75]:


# Load the CSV file
file_path = 'reduced_data.csv'  # Update with the correct path if necessary
df = pd.read_csv(file_path)
df


,severe,LDH,IL6,Neutrophil %,LCR
0,0,501,290.37,59.7,1.65
1,0,645,21.31,90.0,0.47
2,0,1477,4.67,90.0,1.11
3,0,1385,49.20,74.1,14.30
4,0,653,185.36,95.0,0.11
...,...,...,...,...,...
396,0,722,200.21,88.9,0.19
397,0,418,259.97,53.7,1.75
398,0,513,23.78,74.4,2.54
399,0,6705,60.79,87.0,0.36


In [76]:
# Handle categorical column
# Handle categorical column
df.rename(columns = {'severe':'severe_1'}, inplace = True)


In [77]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 401 entries, 0 to 400
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   severe_1      401 non-null    int64  
 1   LDH           401 non-null    int64  
 2   IL6           401 non-null    float64
 3   Neutrophil %  401 non-null    float64
 4   LCR           401 non-null    float64
dtypes: float64(3), int64(2)
memory usage: 15.8 KB


In [78]:


# Ensure all numerical columns are included, assigning default values if necessary
for col in numerical_cols:
    if col not in df.columns:
        df[col] = 0  # or any other default value

# Standardize numerical columns
df[numerical_cols] = scaler.transform(df[numerical_cols])


# Select the top 5 features
df_top5 = df[top_5_features]
df_top5

,LDH,IL6,Neutrophil %,LCR,severe_1
0,-0.383391,0.110233,-1.703031,-0.253921,0
1,-0.098564,-0.522255,0.714805,-0.300430,0
2,1.547102,-0.561371,0.714805,-0.275205,0
3,1.365129,-0.456693,-0.553960,0.244671,0
4,-0.082741,-0.136618,1.113788,-0.314619,0
...,...,...,...,...,...
396,0.053739,-0.101709,0.627029,-0.311466,0
397,-0.547562,0.038771,-2.181810,-0.249980,0
398,-0.359656,-0.516449,-0.530021,-0.218842,0
399,11.887897,-0.429448,0.475416,-0.304766,0


In [82]:

# Make predictions using the loaded model
predictions = svm_model_top5.predict(df_top5)
predictions_proba = svm_model_top5.predict_proba(df_top5)

# Create the output DataFrame
output_df = pd.DataFrame({
    'severe_1': df['severe_1'],
    'predicted_outcome': ['survived' if pred == 0 else 'death' for pred in predictions],
    'probability_survival': [round(prob[0] * 100, 1) for prob in predictions_proba]
})

# Display the output DataFrame
print(output_df)

# Save the output DataFrame to a CSV file
output_df.to_csv('predicted_outcomes_all.csv', index=False)


     severe_1 predicted_outcome  probability_survival
0           0          survived                  93.9
1           0          survived                  93.0
2           0          survived                  93.9
3           0          survived                  94.7
4           0          survived                  93.0
..        ...               ...                   ...
396         0          survived                  92.9
397         0          survived                  94.0
398         0          survived                  93.8
399         0          survived                  75.8
400         0          survived                  93.3

[401 rows x 3 columns]


In [83]:
predictions

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0,
       0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1,
       0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0,
       0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1,
       0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0,
       1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0,
       1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1,